In [9]:
%pylab inline
from datascience import *
import pandas as pd
import numpy as np
import os
import re

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


# Notes

- Changed all [he, she] into [he/ she]
- Changed all [itself, themselves] to [itself/ themselves]

In [10]:
with open('Enmerkar.txt', 'r') as text_file:
    enmerkar = text_file.read()

In [11]:
enmerkar_list = enmerkar.split(':')
enmerkar_list = [word for word in enmerkar_list if '[' in word]

In [12]:
etcsl_words = re.findall(r"(?<=\[)(.*?)(?=\])", enmerkar)
etcsl_words = [word for word in etcsl_words if word != '1']
etcsl_words

['brick(work)',
 '(mountain) land',
 'shining',
 'to go out or in',
 'brick(work)',
 'flat space',
 'lapis lazuli',
 'to go out or in',
 'town',
 'heaven',
 'place',
 'to grow',
 'name',
 'rainbow',
 'heaven',
 'to be adjacent',
 'brilliance',
 'to be multicoloured',
 'heaven',
 'to stand',
 'crescent (moon)',
 'to be new',
 'to be',
 'essence',
 'to be big',
 'princeliness',
 'to erect',
 '(mountain) land',
 'to be pure',
 'day(light)',
 'to be good',
 'place',
 'to place',
 'moon(light)',
 'the Land',
 'to go out or in',
 'day(light)',
 'to be shining',
 'the Land',
 'horn',
 'to equal',
 'rear cow',
 'type of cow',
 'abundance',
 'to go out or in',
 'mouth',
 'to cut',
 '(mountain) land',
 'to approach',
 '(awesome) radiance',
 'bright metal',
 'right',
 'garment',
 'to cover (together)',
 'linen (fibre)',
 'to release',
 'day(light)',
 'day(light)',
 'lord',
 'night',
 'dais',
 'king',
 'minister',
 'lord',
 'minister',
 'name',
 'to be',
 'minister',
 'lord',
 'name',
 'lord',
 't

In [13]:
Enmerkar_table = Table.read_table('Enmerkar.txt', sep = ',')
Enmerkar_table = Enmerkar_table.drop(['text_name', 'etcsl_no'])
Enmerkar_table

l_no,text
1,sux:šeg₁₂[brick(work)]N sux:kur[(mountain) land]N sux:šu ...
2,sux:Kulaba[1]SN sux:iri[town]N sux:an[heaven]N sux:ki[pl ...
3,sux:Unug[1]SN sux:mu[name]N sux:tir-an-na[rainbow]N
4,sux:an[heaven]N sux:us₂[to be adjacent]V sux:si-muš₃[bri ...
5,sux:an[heaven]N sux:gub[to stand]V sux:ud-sakar[crescent ...
6,sux:me[essence]N sux:gal[to be big]V sux:nam-nun[princel ...
7,sux:kur[(mountain) land]N sux:sikil[to be pure]V sux:ud[ ...
8,sux:itid[moon(light)]N sux:kalam[the Land]N sux:e₃[to go ...
9,sux:ud[day(light)]N sux:zalag[to be shining]V sux:kalam[ ...
10,sux:ab₂-ur₃[rear cow]N sux:ab₂-saharₓ[type of cow]N sux: ...


In [17]:
proper_nouns = {
'CN': 'Constellation Name (star)',
'DN': 'Deity Name',
'EN': 'Ethnicity Name',
'FN': 'Field Name',
'GN': 'Geographical Name (for regions and countries)',
'MN': 'Month Name',
'ON': 'Object Name (usually for objects associated with a god)',
'PN': 'Personal Name',
'RN': 'Royal Name',
'SN': 'Settlement Name',
'TN': 'Temple Name',
'WN': 'Water Name',
}

simple_terms = {
    'AJ': 'Adjective',
    'AV': 'Adverb',
    'C': 'Conjunction',
    'N': 'Noun',
    'NU': 'Number',
    'PD': 'Part of Speech',
    'V': 'Verb',
}

In [70]:
def term_finder (line):
    #returns the meanings of words in a line of text
    terms = re.findall(r"(?<=\[)(.*?)(?=\])", line)
    return terms

def proper_noun_finder(line):
    #returns a list of all the proper nouns in a line of text
    nouns = re.findall(r"(?<=\:)(.*?)(?=\[)", line)
    nouns = [word for word in nouns if (len(word) > 1 and word[0].isupper() and not word[1].isupper())]
    return nouns

def speech_article_finder(line, proper_noun_filter = True):
    #returns the speech articles for proper_nouns or all words
    terms = re.findall(r"(?<=\])(.*?)(?=\s)", line)
    if proper_noun_filter:
        articles = [term for term in terms if term in proper_nouns]
    else: 
        articles = terms
    return articles

In [79]:
Enmerkar_table = Enmerkar_table.with_columns([
    'terms', Enmerkar_table.apply(term_finder, 'text'), 
    'proper_nouns', Enmerkar_table.apply(proper_noun_finder, 'text'), 
    'speech_articles', Enmerkar_table.apply(speech_article_finder, 'text')
    ])

Enmerkar_table

l_no,text,terms,proper_nouns,speech_articles
1,sux:šeg₁₂[brick(work)]N sux:kur[(mountain) land]N sux:šu ...,"['brick(work)', '(mountain) land', 'shining', 'to go out ...",[],[]
2,sux:Kulaba[1]SN sux:iri[town]N sux:an[heaven]N sux:ki[pl ...,"['1', 'town', 'heaven', 'place', 'to grow']",['Kulaba'],['SN']
3,sux:Unug[1]SN sux:mu[name]N sux:tir-an-na[rainbow]N,"['1', 'name', 'rainbow']",['Unug'],['SN']
4,sux:an[heaven]N sux:us₂[to be adjacent]V sux:si-muš₃[bri ...,"['heaven', 'to be adjacent', 'brilliance', 'to be multic ...",[],[]
5,sux:an[heaven]N sux:gub[to stand]V sux:ud-sakar[crescent ...,"['heaven', 'to stand', 'crescent (moon)', 'to be new', ' ...",[],[]
6,sux:me[essence]N sux:gal[to be big]V sux:nam-nun[princel ...,"['essence', 'to be big', 'princeliness', 'to erect']",[],[]
7,sux:kur[(mountain) land]N sux:sikil[to be pure]V sux:ud[ ...,"['(mountain) land', 'to be pure', 'day(light)', 'to be g ...",[],[]
8,sux:itid[moon(light)]N sux:kalam[the Land]N sux:e₃[to go ...,"['moon(light)', 'the Land', 'to go out or in']",[],[]
9,sux:ud[day(light)]N sux:zalag[to be shining]V sux:kalam[ ...,"['day(light)', 'to be shining', 'the Land', 'horn', 'to ...",[],[]
10,sux:ab₂-ur₃[rear cow]N sux:ab₂-saharₓ[type of cow]N sux: ...,"['rear cow', 'type of cow', 'abundance', 'to go out or in']",[],[]


In [82]:
Enmerkar_table

l_no,text,terms,proper_nouns,speech_articles
1,sux:šeg₁₂[brick(work)]N sux:kur[(mountain) land]N sux:šu ...,"['brick(work)', '(mountain) land', 'shining', 'to go out ...",[],[]
2,sux:Kulaba[1]SN sux:iri[town]N sux:an[heaven]N sux:ki[pl ...,"['1', 'town', 'heaven', 'place', 'to grow']",['Kulaba'],['SN']
3,sux:Unug[1]SN sux:mu[name]N sux:tir-an-na[rainbow]N,"['1', 'name', 'rainbow']",['Unug'],['SN']
4,sux:an[heaven]N sux:us₂[to be adjacent]V sux:si-muš₃[bri ...,"['heaven', 'to be adjacent', 'brilliance', 'to be multic ...",[],[]
5,sux:an[heaven]N sux:gub[to stand]V sux:ud-sakar[crescent ...,"['heaven', 'to stand', 'crescent (moon)', 'to be new', ' ...",[],[]
6,sux:me[essence]N sux:gal[to be big]V sux:nam-nun[princel ...,"['essence', 'to be big', 'princeliness', 'to erect']",[],[]
7,sux:kur[(mountain) land]N sux:sikil[to be pure]V sux:ud[ ...,"['(mountain) land', 'to be pure', 'day(light)', 'to be g ...",[],[]
8,sux:itid[moon(light)]N sux:kalam[the Land]N sux:e₃[to go ...,"['moon(light)', 'the Land', 'to go out or in']",[],[]
9,sux:ud[day(light)]N sux:zalag[to be shining]V sux:kalam[ ...,"['day(light)', 'to be shining', 'the Land', 'horn', 'to ...",[],[]
10,sux:ab₂-ur₃[rear cow]N sux:ab₂-saharₓ[type of cow]N sux: ...,"['rear cow', 'type of cow', 'abundance', 'to go out or in']",[],[]


In [86]:
def term_correlater(line):
    line_nouns = [key for key in line if key in proper_nouns]
    return(names[row_num])

In [22]:
noun_dict = Table.from_columns_dict(proper_nouns)


/opt/conda/lib/python3.4/site-packages/datascience/tables.py:107: FutureWarning: Table.from_columns_dict is deprecated. Use Table().with_columns(...)
  warnings.warn("Table.from_columns_dict is deprecated. Use Table().with_columns(...)", FutureWarning)


In [ ]:
pronoun_list = []
for line in Enmerkar_table['text']:
    x = re.findall(r"([A-Z]\w+)", line)
    if len(x)%2 == 0:
        pronoun_list.append(x)
        
for group in pronoun_list:
    labels = group[1::2]
    nouns = group[0::2]
    if len(labels) > 1:
        for label in labels:
            noun_dict[label] #add them in order... same issue
    else:
        noun_dict[labels[0]] = nouns[0] 
    
    

In [ ]:
def partitioning(line_no):
    
    ln = int(''.join(c for c in line_no if c.isdigit()))

    if(ln <= 13):
        return "1.1"
    elif (ln <= 21):
        return "1.2"
    
    elif (ln <= 39):
        return "2.1.1"
    elif (ln <= 51):
        return "2.1.2"
    elif (ln <= 69):
        return "2.1.3"
    
    elif (ln <= 76):
        return "2.2.1"
    elif (ln <= 90):
        return "2.2.2"
    elif (ln <= 113):
        return "2.2.3"
    
    elif (ln <= 127):
        return "2.3.1"
    elif (ln <= 132):
        return "2.3.2"
    elif (ln <= 134):
        return "2.3.3"
    
    elif (ln <= 138):
        return "3.1.1"
    elif (ln <= 149):
        return "3.1.2"
    elif (ln <= 162):
        return "3.1.3"
    elif (ln <= 169):
        return "3.1.4"
    
    elif (ln <= 184):
        return "3.2.1"
    elif (ln <= 197):
        return "3.2.2"
    elif (ln <= 205):
        return "3.2.3"
    elif (ln <= 210):
        return "3.2.4"
    elif (ln <= 221):
        return "3.2.5"
    
    elif (ln <= 227):
        return "4.1"
    
    elif (ln <= 248):
        return "4.2.1"
    elif (ln <= 254):
        return "4.2.2"
    elif (ln <= 263):
        return "4.2.3"
    elif (ln <= 273):
        return "4.2.4"
    
    elif (ln <= 280):
        return "5.1"
    elif (ln <= 283):
        return "5.2"
    elif (ln <= 310):
        return "B"
    return "0"

def small_partition(line_no):
    ln = int(''.join(c for c in line_no if c.isdigit()))
    if(ln <= 13):
        return "1.1"
    elif (ln <= 21):
        return "1.2"
    elif (ln <= 69):
        return "2.1"
    elif (ln <= 113):
        return "2.2"
    elif (ln <= 134):
        return "2.3"
    elif (ln <= 169):
        return "3.1"
    elif (ln <= 221):
        return "3.2"
    elif (ln <= 227):
        return "4.1"
    elif (ln <= 273):
        return "4.2"
    elif (ln <= 280):
        return "5.1"
    elif (ln <= 283):
        return "5.2"
    elif (ln <= 310):
        return "6"
    return "0"

In [ ]:
Enmerkar_table.append_column('section', Enmerkar_table.apply(small_partition, 0))
Enmerkar_graph = Enmerkar_table.select([3, 4]).group('section', list)
Enmerkar_graph

In [ ]:
def list_flattening(pn_list):
    return [noun for nouns in pn_list for noun in nouns]

Enmerkar_graph.append_column('proper nouns', Enmerkar_graph.apply(list_flattening, 1))
Enmerkar_graph

In [ ]:
from collections import Counter
proper_nouns_list = set(list_flattening(Enmerkar_graph.column('proper nouns')))

rows = []
for i in np.arange(0, Enmerkar_graph.num_rows):
    counts = []
    for noun in proper_nouns_list:
        counts.append(Enmerkar_graph['proper nouns'][i].count(noun))
    rows.append(list(np.array(counts).cumsum()))
    
Enmerkar_plot = Table(proper_nouns_list).with_rows(rows)
Enmerkar_plot.append_column('section', Enmerkar_graph.column('section'))
Enmerkar_plot.move_to_start('section')
Enmerkar_plot

In [ ]:
Enmerkar_plot.plot('section', overlay=True)

# 